In [ ]:
# purpose: use that subject's last run for the first run so their data could be combined in one training SRM



In [1]:
import logging
logging.basicConfig(level=logging.DEBUG, format='%(levelname)s:%(asctime)s - %(message)s')
import numpy as np
import pickle
import nibabel
import nilearn
from nilearn.image import resample_to_img
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn.plotting import show
from nilearn.plotting import plot_roi
from nilearn import image
from nilearn.masking import apply_mask
# get_ipython().magic('matplotlib inline')
import scipy
import matplotlib
import matplotlib.pyplot as plt
from nilearn import image
from nilearn.input_data import NiftiMasker
#from nilearn import plotting
import nibabel
from nilearn.masking import apply_mask
from nilearn.image import load_img
from nilearn.image import new_img_like
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets, svm, metrics
from sklearn.linear_model import Ridge
from sklearn.svm import SVC, LinearSVC
from sklearn.cross_validation import KFold
from sklearn.cross_validation import LeaveOneLabelOut
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_selection import SelectFwe
from scipy import signal
from scipy.fftpack import fft, fftshift
from scipy import interp
import csv
params = {'legend.fontsize': 'large',
          'figure.figsize': (5, 3),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
font = {'weight': 'bold',
        'size': 22}
plt.rc('font', **font)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectPercentile, f_classif, GenericUnivariateSelect, SelectKBest, chi2
from sklearn.feature_selection import RFE
import os
import seaborn as sns
import pandas as pd
import csv
from scipy import stats
import brainiak
import brainiak.funcalign.srm
import sys
from sklearn.utils import shuffle
import random
from datetime import datetime
random.seed(datetime.now())

Could not import jpeg_ls
DEBUG:2019-02-27 14:49:56,961 - Could not import jpeg_ls
Could not import gdcm
DEBUG:2019-02-27 14:49:56,965 - Could not import gdcm
/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
DEBUG:2019-02-27 14:49:58,887 - $HOME=/usr/people/amennen
DEBUG:2019-02-27 14:49:58,890 - matplotlib data path /jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/matplotlib/mpl-data
DEBUG:2019-02-27 14:49:58,895 - loaded rc file /jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:2019-02-27 14:49:58,896 - matplotlib version 2.2.2
DEBUG:2019-02-27 14:49:58,897 - interactive is False
DEBUG:2019-02-27 14:49:58,897 - platform is linux
DEBUG:2019-02-27 14:49:58,898 - loaded modules:

DEBUG:2019-02-27 14:49:58,970 - CACHEDIR=/usr/people/amennen/.cache/matplotlib
DEBUG:2019-02-27 14:49:59,009 - Using fontManager instance from /usr/people/amennen/.cache/matplotlib/fontList.json
DEBUG:2019-02-27 14:49:59,259 - backend module://ipykernel.pylab.backend_inline version unknown
/jukebox/pkgs/PYGER/beta/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
DEBUG:2019-02-27 14:50:00,620 - backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
# first get that subject's data ready for preprocessing in the same way
projectDir = '/jukebox/norman/amennen/RT_prettymouth/data/bids/Norman/Mennen/5516_greenEyes/'
fmriprep_dir=projectDir + '/derivatives/fmriprep'
preproc_dir = projectDir + '/derivatives/preproc'
TOM_large = '/jukebox/norman/amennen/prettymouth_fmriprep2/ROI/TOM_large_resampled_maskedbybrain.nii.gz'
TOM_cluster = '/jukebox/norman/amennen/prettymouth_fmriprep2/ROI/TOM_cluster_resampled_maskedbybrain.nii.gz'
fmriprep2_path = '/jukebox/norman/amennen/prettymouth_fmriprep2/code'
offline_path = '/jukebox/norman/amennen/prettymouth_fmriprep2/code/saved_classifiers'

In [3]:
# steps:
# -- 1. load in data, mask properly
# -- 2. (if needed) truncate and filter
# -- 3. otherwise, just load and mask -- this cell
subjectNum=101
bids_id = 'sub-{0:03d}'.format(subjectNum)
ses_id = 'ses-{0:02d}'.format(1)
data_dir = fmriprep_dir + '/' + bids_id + '/' + ses_id + '/' + 'func'
story_TR_1 = 14
story_TR_2 = 464
run_TRs = 450
nRuns = 4 # for subject 101 nRuns = 3, for subject 102 nRuns = 4

if subjectNum == 101:
    nRuns = 3
cheating_prob = np.zeros((run_TRs,nRuns))


In [ ]:
# now use the last run